In [1]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from pypdf import PdfReader
load_dotenv()
import os

In [2]:
MODEL = 'gpt-3.5-turbo'

In [3]:
model = ChatOpenAI(
    openai_api_key = os.environ["OPENAI_API_KEY"],
    model=MODEL
)

/home/hamza/miniconda3/envs/chatbot/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
reader = PdfReader("/home/hamza/Documents/RAG/documents/crime-and-punishment.pdf")
text = ""
texts = []
parts = [{"min":6, "max":132}, {"min":132, "max":279}, {"min":280, "max":396}, {"min":397, "max":507}, {"min":508, "max":612}, {"min":612, "max":744}]
for j in range(len(parts)):
    text = ""
    for i in range(parts[j]['min'], parts[j]['max']):
        page = reader.pages[i]
        text+=page.extract_text()
    texts.append(text)


In [5]:
text_spliter = RecursiveCharacterTextSplitter(
   chunk_size = 50000,
   chunk_overlap=0
)

In [6]:
import time
summaries = ''

for j in range(len(texts)):
    summary= ''
    content = text_spliter.split_text(texts[j])
    docs = [Document(page_content=t) for t in content]
    for i in range(len(docs)):
        prompt = f"write a comprehensive summary for : {docs[i].page_content}"


        from langchain.schema import (
            HumanMessage
        )

        messages = [
            HumanMessage(content=prompt),
        ]
        res = model(messages)
        summary+=res.content
    summaries+=summary
    time.sleep(20)

/home/hamza/miniconda3/envs/chatbot/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [16]:
prompt = f"write a comprehensive summary of 2000 from these summaries of parts of the book: {summaries}"


from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    HumanMessage(content=prompt),
]
res = model(messages)

In [8]:
prompt

'write a comprehensive summary of two thousands words from these summaries of parts of the book: rsation leads Raskolnikov to contemplate the idea of committing a crime to obtain the old woman\'s money and use it for the greater good of society.\n\nThe passage delves into Raskolnikov\'s inner turmoil, his moral struggles, and his justification for potentially committing a heinous act. It highlights his conflicting thoughts and the internal battle between his desire to do good and his darker impulses. Ultimately, it sets the stage for the pivotal decision that Raskolnikov will have to make regarding the old woman and her money.The passage describes Raskolnikov\'s descent into madness and desperation as he commits two brutal murders. The first is of an old pawnbroker woman, who he justifies as being harmful to society and therefore deserving of death. He then kills her sister, Lizaveta, who stumbles upon the scene. Raskolnikov\'s actions are methodical and calculated, but he begins to pa

In [18]:
len(res.content)

1719

In [17]:
res.content

'In the novel "Crime and Punishment" by Fyodor Dostoevsky, the year 2000 is depicted through a series of interconnected events and character interactions that highlight the themes of guilt, redemption, and the consequences of one\'s actions. The protagonist, Raskolnikov, grapples with inner turmoil and conflicting emotions as he struggles to come to terms with the murders he has committed. The passage explores Raskolnikov\'s moral struggles, paranoia, and sense of isolation as he navigates the aftermath of his actions.\n\nThroughout the text, Raskolnikov is confronted with his own guilt and the weight of his crimes, leading to moments of introspection and self-reflection. The dream sequence he experiences serves as a catalyst for his inner turmoil, pushing him to confront his moral compass and the consequences of his actions. As he interacts with various characters, such as Porfiry Petrovitch, Sonia, and Katerina Ivanovna, Raskolnikov is forced to confront his own beliefs and grapple w

In [11]:
len(summaries)

32280

In [20]:
!pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=90dac6980d0c76e75e3dfb6b0ae272254947c38b24b4d0fd33c1fe473666c6c1
  Stored in directory: /home/hamza/.cache/pip/wheels/44/35/8b/86ce00cec7e4d13c5f189680ae0fa82f919bedc066c2cddae9
Successfully built fpdf


In [26]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

# Function to convert text to PDF
def text_to_pdf(text, output_filename):
    # Create a PDF document
    pdf = SimpleDocTemplate(output_filename, pagesize=letter)
    
    # Create a list of paragraphs
    styles = getSampleStyleSheet()
    paragraphs = [Paragraph(text, styles["Normal"])]
    
    # Add paragraphs to the PDF document
    pdf.build(paragraphs)

# Sample text
sample_text = res.content

# Convert text to PDF
output_filename = "summary.pdf"
text_to_pdf(sample_text, output_filename)

print(f"PDF saved as '{output_filename}'")


PDF saved as 'text_to_pdf.pdf'
